# Análisis de Datos - Software de Planta Industrial
En este notebook se realiza un análisis sobre datos extraídos de un software utilizado en diferentes procesos de una planta fabril.

Se pretende obtener un resumen por operario que contenga los siguientes datos:
- Horas totales en el mes.
- Horas en cada proceso productivo, otras actividades y/o paradas.
- Rendimiento en cada proceso productivo.

Por otro lado, se realiza un análisis del proceso de "Corte de Ladrillos" observando la producción a lo largo del mes y la variación en el rendimiento por operario.

## Importar librerías

In [ ]:
from sqlalchemy import text
import pandas as pd
import matplotlib.pyplot as plt
from db_utils import get_engine

## Conectar a Base de Datos

In [ ]:
engine = get_engine()

## Análisis de Operarios

### Cargar datos de operarios

In [ ]:
min_procesos = 2

query = text('''
SELECT 
    u.id_usuario AS ID,
    u.nombre AS NOMBRE,
    u.apellido AS APELLIDO,
    COUNT(DISTINCT p.origen) AS PROCESOS
FROM 
    usuarios u
JOIN (
        SELECT
            id_usuario,
            'pprod_cortelad' AS origen 
        FROM pprod_cortelad

        UNION

        SELECT
            id_usuario,
            'pprod_inyeccion' AS origen 
        FROM pprod_inyeccion

        UNION

        SELECT
            id_usuario,
            'pprod_embolsado' AS origen 
        FROM pprod_embolsado
    ) p 
    ON u.id_usuario = p.id_usuario
GROUP BY 
    u.id_usuario, 
    u.nombre
HAVING COUNT(DISTINCT p.origen) >= :min_procesos
ORDER BY PROCESOS desc;
''')

df = pd.read_sql(query, engine, params={"min_procesos": min_procesos})
print(df)

## Análisis exploratorio

In [ ]:
print(df.head())
print(df.describe())

## Visualización

In [ ]:
plt.figure(figsize=(10,5))
df.groupby("fecha")["ventas"].sum().plot()
plt.title("Ventas diarias")
plt.xlabel("Fecha")
plt.ylabel("Total ventas")
plt.show()